In [9]:
# Install required packages
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [10]:
#dotenv module
from dotenv import load_dotenv
_ = load_dotenv()

In [11]:
from typing import Annotated
from typing_extensions import TypedDict


In [12]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("google_genai:gemini-2.0-flash")

In [13]:
from langchain_tavily import TavilySearch

tool = TavilySearch(max_results = 2)
tools = [tool]

In [14]:

from langchain_core.messages import BaseMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition


In [15]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)


llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools= [tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)

# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")
graph = graph_builder.compile()

In [ ]:
# We can easily display the graph we have created
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
result = graph.invoke({"messages": ["What is the weather in sf?"]})
print(result['messages'][-1].content)

I do not have the capability to get the weather. I can only use the tavily_search tool to search for information.
